In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_val_predict

# Modelling Algorithms
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier , GradientBoostingClassifier

# Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab

# Configure visualisations
%matplotlib inline

In [ ]:
df = pd.read_csv("../datasets/winemag-data_first150k.csv", index_col=0)
df.head()

# Task:
### Make a model which will predict whether a score of a given wine ("points") will be over 90

## 1. Dataset preprocessing (preparation) 

### 1.1 
<i>Convert a variable "points" into a categorical 1 and 0 which is necessary for a classification algorithms<br>
1 - "points" are higher than 90 points<br>
0 - "points" are lower or equal 90 points</i>

In [ ]:
df["points"] = np.where(df['points'] > 90, 1, 0)
df["points"].head()

### 1.2 
<i>Replace NaN with "Unknown"<br> 
In case of "price" is NaN we will drop such object.<br> 
We will do that because we need to convert all prices of objects to numbers in order to use this variable as a target variable for applying classification algorithms.<br> 
And with the price its uncertain with which price to replace NaN, so we will just drop such objects.</i>

In [ ]:
df["price"] = df["price"].dropna()
df = df[pd.notnull(df["price"])]

df = df.replace(np.nan, 'Unknown', regex=True)
df.head(50)

### 1.3 
<i>Convert all string variables to their categorical equivalents</i>

#### country

In [ ]:
df["country_codes"] = df["country"].astype("category").cat.codes
df["country_codes"].head()

In [ ]:
for country, code in zip(range(0, len(df["country"].astype("category").cat.categories)),
                         df["country"].astype("category").cat.categories):
    print("{0}:{1}".format(country, code))

In [ ]:
df.head()

#### designation

In [ ]:
Counter(df["designation"]).most_common(5)

We can see that "designation" values are repeating. We can replace than with categories codes.

In [ ]:
df["designation_codes"] = df["designation"].astype("category").cat.codes
df["designation_codes"].head()

#### province

In [ ]:
Counter(df["province"]).most_common(5)

We can see that "province" values are repeating. We can replace than with categories codes.

In [ ]:
df["province_codes"] = df["province"].astype("category").cat.codes
df["province_codes"].head()

#### region_1

In [ ]:
Counter(df["region_1"]).most_common(5)

We can see that "region_1" values are repeating. We can replace than with categories codes.

In [ ]:
df["region_1_codes"] = df["region_1"].astype("category").cat.codes
df["region_1_codes"].head()

#### region_2

In [ ]:
Counter(df["region_2"]).most_common(5)

We can see that "region_2" values are repeating. We can replace than with categories codes.

In [ ]:
df["region_2_codes"] = df["region_2"].astype("category").cat.codes
df["region_2_codes"].head()

#### variety

In [ ]:
Counter(df["variety"]).most_common(5)

We can see that "variety" values are repeating. We can replace than with categories codes.

In [ ]:
df["variety_codes"] = df["variety"].astype("category").cat.codes
df["variety_codes"].head()

#### winery

In [ ]:
Counter(df["winery"]).most_common(5)

We can see that "winery" values are repeating. We can replace than with categories codes.

In [ ]:
df["winery_codes"] = df["winery"].astype("category").cat.codes
df["winery_codes"].head()

In [ ]:
df_new = df[["price","country_codes",
                         "designation_codes","province_codes",
                         "region_1_codes","region_2_codes",
                         "variety_codes","winery_codes", "points"]]
df_new.head(10)

## Part 2. 
### Train/test dataset splitting
<i>Let's split our dataset into training and test datasets and also select a target variable ("price") from test and training sets.
Object with ids 2, 7, 12, ... go to the test dataset.<br>
All others - into training dataset.</i>

In [ ]:
train_df = df_new[(df_new.index + 3) % 5 != 0]
train_df.head(10)

In [ ]:
test_df = df_new[(df_new.index + 3) % 5 == 0]
test_df.head(10)

In [ ]:
train_y = train_df["points"]
train_y.head(10)

In [ ]:
test_y = test_df["points"]
test_y.head(10)

In [ ]:
train_df = train_df.drop('points', 1)
test_df = test_df.drop('points', 1)

## Part 3. 
### Building a classification model

### 3.1
<i>Training models with different algorithms</i>

State a "random_state" explicitly for reproducibility of experiments.

In [ ]:
my_rand_state = 241

### DecisionTreeClassifier

In [ ]:
decis_tree = DecisionTreeClassifier()
decis_tree.fit(train_df, train_y)

### KNeighborsClassifier

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=4)
neigh.fit(train_df, train_y)

### LogisticRegression

In [ ]:
log_regr = LogisticRegression()
log_regr.fit(train_df, train_y)

### GaussianNB

In [ ]:
gauss = GaussianNB()
gauss.fit(train_df, train_y)

### RandomForestClassifier

In [ ]:
random_forest = RandomForestClassifier()
random_forest.fit(train_df, train_y)

### GradientBoostingClassifier

In [ ]:
grad_boost = GradientBoostingClassifier()
grad_boost.fit(train_df, train_y)

### 3.2
<i>Outputting accuracy scores</i>

### DecisionTreeClassifier

In [ ]:
accuracy_score(test_y, decis_tree.predict(test_df))

### KNeighborsClassifier

In [ ]:
accuracy_score(test_y, neigh.predict(test_df))

### LogisticRegression

In [ ]:
accuracy_score(test_y, log_regr.predict(test_df))

### GaussianNB

In [ ]:
accuracy_score(test_y, gauss.predict(test_df))

### RandomForestClassifier

In [ ]:
accuracy_score(test_y, random_forest.predict(test_df))

### GradientBoostingClassifier

In [ ]:
accuracy_score(test_y, grad_boost.predict(test_df))

#### The best accuracy with hyperparameters by defaults showed models "RandomForestClassifier" and "GradientBoostingClassifier"

### 3.3 
<i>Show variables that have the most influence on the correctness of predictions.</i>

In [ ]:
def plot_model_var_imp( model , X , y, title ):
    imp = pd.DataFrame( 
        model.feature_importances_  , 
        columns = [ 'Importance' ] , 
        index = X.columns 
    )
    imp = imp.sort_values( [ 'Importance' ] , ascending = True )
    imp[ : 10 ].plot( kind = 'barh', title=title )

plot_model_var_imp( decis_tree , train_df , train_y, title="Desicion Tree" )
plot_model_var_imp( random_forest , train_df , train_y, title="Random forest" )
plot_model_var_imp( grad_boost , train_df , train_y, title="Gradient Boosting" )

#### On example of 3 algorithms we can see that the most important variable for a predictions is "price".